In [14]:
from viresclient import SwarmRequest
from viresclient import ClientConfig
import datetime as dt
import sys,os
from pathlib import Path
import pandas as pd
import numpy as np
import datetime, time
from datetime import timedelta
import math
import pathlib
from datetime import datetime
import time
import calendar
import datetime

import chaosmagpy as cp
from chaosmagpy import load_CHAOS_matfile
from chaosmagpy.data_utils import mjd2000
from gg_to_geo import gg_to_geo

In [4]:
VirESToken = "rp-foC8gz-VTFz7He6SMPnQtKEpasEaB"

## Read the GPS Track

In [5]:
#Enter the name of your GPS track csv file including the extension .csv  and press Enter (e.g. BirdGPSTrajectory.csv)
gpsfilename=input()
Lat=input()
Long=input()
DateTime=input()

BirdGPSTrajectory.csv
location-lat
location-long
timestamp


In [6]:
os.chdir(r"./data")
def getGPSData():
    nfp = pd.read_csv(gpsfilename, parse_dates=[0], encoding='utf-8', dayfirst=True, usecols=[Lat, Long, DateTime])
    return nfp

In [7]:
GPSData = getGPSData()
GPSData.rename(columns={Lat: 'gpsLat', Long: 'gpsLong', DateTime: 'gpsDateTime'}, inplace = True)
GPSData

,gpsDateTime,gpsLong,gpsLat
0,2014-09-08 05:54:00,68.307333,70.854717
1,2014-09-08 06:10:00,67.975050,70.830300
2,2014-09-08 06:26:00,67.752417,70.761717
3,2014-09-08 06:42:00,67.561983,70.686517
4,2014-09-08 07:14:00,67.548317,70.685450
...,...,...,...
968,2014-10-01 10:37:00,22.509733,55.214750
969,2014-10-03 11:28:00,7.292517,53.392167
970,2014-10-04 07:11:00,5.922067,53.107067
971,2014-10-04 14:12:00,5.811383,53.014967


In [9]:
#2.1 Convert the gpsDateTime to datetime python object
GPSData['gpsDateTime'] = pd.to_datetime(GPSData['gpsDateTime'], format='%d/%m/%Y %H:%M:%S')
GPSData['gpsDateTime'] = GPSData['gpsDateTime'].map(lambda x: x.replace(second=0))
GPSData['gpsLat'] = GPSData['gpsLat'].astype(float)
#2.2 Adding new column epoch, will be usefuel to compare the date&time o each gps point againts the gathered swmarm data points
GPSData['epoch'] = GPSData['gpsDateTime'].astype('int64')//1e9
GPSData['epoch'] = GPSData['epoch'].astype(int)
#2.3 Computing Date and Time columns
GPSData['dates'] = GPSData['gpsDateTime'].dt.date
GPSData['times'] = GPSData['gpsDateTime'].dt.time
#2.4 Maximum and Minimum values
MinDateTime = min(GPSData['gpsDateTime'])
MaxDateTime = max(GPSData['gpsDateTime'])
dt_object1 = MinDateTime.to_pydatetime()
dt_object2 = MaxDateTime.to_pydatetime()
startDate = MinDateTime.date()
endDate = MaxDateTime.date()
MinTime=MinDateTime.time()
MaxTime=MaxDateTime.time()
GPSData

,gpsDateTime,gpsLong,gpsLat,epoch,dates,times
0,2014-09-08 05:54:00,68.307333,70.854717,1410155640,2014-09-08,05:54:00
1,2014-09-08 06:10:00,67.975050,70.830300,1410156600,2014-09-08,06:10:00
2,2014-09-08 06:26:00,67.752417,70.761717,1410157560,2014-09-08,06:26:00
3,2014-09-08 06:42:00,67.561983,70.686517,1410158520,2014-09-08,06:42:00
4,2014-09-08 07:14:00,67.548317,70.685450,1410160440,2014-09-08,07:14:00
...,...,...,...,...,...,...
968,2014-10-01 10:37:00,22.509733,55.214750,1412159820,2014-10-01,10:37:00
969,2014-10-03 11:28:00,7.292517,53.392167,1412335680,2014-10-03,11:28:00
970,2014-10-04 07:11:00,5.922067,53.107067,1412406660,2014-10-04,07:11:00
971,2014-10-04 14:12:00,5.811383,53.014967,1412431920,2014-10-04,14:12:00


## Get Unique Dates

In [10]:
datestimeslist = []
for index, row in GPSData.iterrows():
    datetimerow  = row['gpsDateTime']
    daterow = row['dates']
    hourrow = row['times']
    hourrow = hourrow.strftime('%H:%M:%S')
    if hourrow < '04:00:00':
        date_bfr = daterow - (timedelta(days=1))
        datestimeslist.append(daterow)
        datestimeslist.append(date_bfr)
    if hourrow > '20:00:00':
        Date_aft = daterow + (timedelta(days=1))
        datestimeslist.append(daterow)
        datestimeslist.append(Date_aft)  
    else:
        datestimeslist.append(daterow)

In [11]:
def uniquelistdates(list): 
    x = np.array(list) 
    uniquelist = np.unique(x)
    return uniquelist

uniquelist_dates = uniquelistdates(datestimeslist)
uniquelist_dates

array([datetime.date(2014, 8, 18), datetime.date(2014, 8, 19),
       datetime.date(2014, 8, 25), datetime.date(2014, 8, 26),
       datetime.date(2014, 8, 27), datetime.date(2014, 8, 28),
       datetime.date(2014, 8, 29), datetime.date(2014, 8, 30),
       datetime.date(2014, 8, 31), datetime.date(2014, 9, 3),
       datetime.date(2014, 9, 5), datetime.date(2014, 9, 6),
       datetime.date(2014, 9, 8), datetime.date(2014, 9, 13),
       datetime.date(2014, 9, 14), datetime.date(2014, 9, 15),
       datetime.date(2014, 9, 17), datetime.date(2014, 9, 18),
       datetime.date(2014, 9, 19), datetime.date(2014, 9, 20),
       datetime.date(2014, 9, 21), datetime.date(2014, 9, 22),
       datetime.date(2014, 9, 23), datetime.date(2014, 9, 24),
       datetime.date(2014, 9, 25), datetime.date(2014, 9, 26),
       datetime.date(2014, 9, 27), datetime.date(2014, 9, 29),
       datetime.date(2014, 9, 30), datetime.date(2014, 10, 1),
       datetime.date(2014, 10, 2), datetime.date(2014, 10, 

## Get Residuals from Swarm

In [12]:
# 1. Every Day, Get the Swarm Data and Residuals: Get_Swarm_and_residuals
# Filter the Bad Ones
# Input:  Time variables
# Output: Swarm DF, including the residuals, ABC combined.

def Get_Swarm_residuals(startDateTime, endDateTime):
    requestA = SwarmRequest() 
    requestB = SwarmRequest()
    requestC = SwarmRequest()
    ### Request for Sat A
    requestA.set_collection("SW_OPER_MAGA_LR_1B")
    requestA.set_products(
        measurements=[
            'F', #Magnetic intensity
            'B_NEC', #The Magnetic values are in NEC system (North, East, Centre)

        ],

        models = [
            
            '"CHAOS_MCO_MLI_MMA" = "CHAOS-Core" + "CHAOS-Static" + "CHAOS-MMA-Primary" + "CHAOS-MMA-Secondary"'
        ],
        residuals=True, #Brining the residuals.
        sampling_step="PT60S", #Get the data every 60 seconds. 
    )
    
    #Define an array to store the data request for Satellite A, based on the start Date and time.
    #You can display dsA to get an idea of how the data is requested.
    dsA = requestA.get_between(
        start_time=startDateTime,
        end_time=endDateTime
    ).as_dataframe()
           
    ### End Request for Sat A
    
    ### Request for Sat B
    requestB.set_collection("SW_OPER_MAGB_LR_1B")
    requestB.set_products(
        measurements=[
            'F', #Magnetic intensity
            'B_NEC',

        ],
        models = [
            
            '"CHAOS_MCO_MLI_MMA" = "CHAOS-Core" + "CHAOS-Static" + "CHAOS-MMA-Primary" + "CHAOS-MMA-Secondary"'
        ],
        residuals=True, 
        sampling_step="PT60S",
    )

    dsB = requestB.get_between(
        start_time= startDateTime,
        end_time= endDateTime
    ).as_dataframe()
    ### End Request for Sat B
    ### Request for Sat C
    requestC.set_collection("SW_OPER_MAGC_LR_1B")
    requestC.set_products(
        measurements=[
            'F',
            'B_NEC',

        ],
        models = [
            
            '"CHAOS_MCO_MLI_MMA" = "CHAOS-Core" + "CHAOS-Static" + "CHAOS-MMA-Primary" + "CHAOS-MMA-Secondary"'
        ],
        residuals=True,
        sampling_step="PT60S",
    )

    dsC = requestC.get_between(
        start_time=startDateTime,
        end_time=endDateTime
    ).as_dataframe()
    ### End Request for Sat C
    dsA[['N_res','E_res','C_res']] = pd.DataFrame(dsA.B_NEC_res_CHAOS_MCO_MLI_MMA.tolist(), index= dsA.index)
    dsB[['N_res','E_res','C_res']] = pd.DataFrame(dsB.B_NEC_res_CHAOS_MCO_MLI_MMA.tolist(), index= dsB.index)
    dsC[['N_res','E_res','C_res']] = pd.DataFrame(dsC.B_NEC_res_CHAOS_MCO_MLI_MMA.tolist(), index= dsC.index)
    dsA_filtered = dsA[dsA['F_res_CHAOS_MCO_MLI_MMA'].between(-2000, 2000)]
    dsB_filtered = dsB[dsB['F_res_CHAOS_MCO_MLI_MMA'].between(-2000, 2000)]
    dsC_filtered = dsC[dsA['F_res_CHAOS_MCO_MLI_MMA'].between(-2000, 2000)]

    dsA_filtered['epoch'] = dsA_filtered.index
    dsA_filtered['timestamp'] = dsA_filtered.index
    dsA_filtered['epoch'] = dsA_filtered['epoch'].astype('int64')//1e9
    dsA_filtered['epoch'] = dsA_filtered['epoch'].astype(int)
    dsA_filtered.set_index("epoch", inplace=True)
    
    dsB_filtered['epoch'] = dsB_filtered.index
    dsB_filtered['timestamp'] = dsB_filtered.index
    dsB_filtered['epoch'] = dsB_filtered['epoch'].astype('int64')//1e9
    dsB_filtered['epoch'] = dsB_filtered['epoch'].astype(int)
    dsB_filtered.set_index("epoch", inplace=True)
    
    dsC_filtered['epoch'] = dsC_filtered.index
    dsC_filtered['timestamp'] = dsC_filtered.index
    dsC_filtered['epoch'] = dsC_filtered['epoch'].astype('int64')//1e9
    dsC_filtered['epoch'] = dsC_filtered['epoch'].astype(int)
    dsC_filtered.set_index("epoch", inplace=True)

    return dsA_filtered, dsB_filtered, dsC_filtered

In [15]:
%%time

hours_t_day = 24
hours_added = datetime.timedelta(hours = hours_t_day)

listdfa = []
listdfb = []
listdfc = []

cc = ClientConfig()
cc.set_site_config("https://vires.services/ows", token=VirESToken)
cc.default_url = "https://vires.services/ows"
cc.save()

for d in uniquelist_dates:
    print("Process for:",d,":" )
    startdate = datetime.datetime.combine(d, datetime.datetime.min.time())
    enddate = startdate + hours_added
    SwarmResidualsA,SwarmResidualsB,SwarmResidualsC = Get_Swarm_residuals(startdate, enddate)
    listdfa.append(SwarmResidualsA)
    listdfb.append(SwarmResidualsB)
    listdfc.append(SwarmResidualsC)

Process for: 2014-08-18 :


[1/1] Processing:  100%|████████████████████████████████████████████████████████|  [ Elapsed: 00:02, Remaining: 00:00 ]
      Downloading: 100%|██████████████████████████████████████████████|  [ Elapsed: 00:00, Remaining: 00:00 ] (0.106MB)
[1/1] Processing:  100%|████████████████████████████████████████████████████████|  [ Elapsed: 00:02, Remaining: 00:00 ]
      Downloading: 100%|██████████████████████████████████████████████|  [ Elapsed: 00:00, Remaining: 00:00 ] (0.106MB)
[1/1] Processing:  100%|████████████████████████████████████████████████████████|  [ Elapsed: 00:02, Remaining: 00:00 ]
      Downloading: 100%|██████████████████████████████████████████████|  [ Elapsed: 00:00, Remaining: 00:00 ] (0.106MB)


Process for: 2014-08-19 :


[1/1] Processing:  100%|████████████████████████████████████████████████████████|  [ Elapsed: 00:02, Remaining: 00:00 ]
      Downloading: 100%|██████████████████████████████████████████████|  [ Elapsed: 00:00, Remaining: 00:00 ] (0.106MB)
[1/1] Processing:  100%|████████████████████████████████████████████████████████|  [ Elapsed: 00:02, Remaining: 00:00 ]
      Downloading: 100%|██████████████████████████████████████████████|  [ Elapsed: 00:00, Remaining: 00:00 ] (0.106MB)
[1/1] Processing:  100%|████████████████████████████████████████████████████████|  [ Elapsed: 00:02, Remaining: 00:00 ]
      Downloading: 100%|██████████████████████████████████████████████|  [ Elapsed: 00:00, Remaining: 00:00 ] (0.106MB)


Process for: 2014-08-25 :


...

Process for: 2014-10-23 :


[1/1] Processing:  100%|████████████████████████████████████████████████████████|  [ Elapsed: 00:02, Remaining: 00:00 ]
      Downloading: 100%|██████████████████████████████████████████████|  [ Elapsed: 00:00, Remaining: 00:00 ] (0.106MB)
[1/1] Processing:  100%|████████████████████████████████████████████████████████|  [ Elapsed: 00:02, Remaining: 00:00 ]
      Downloading: 100%|██████████████████████████████████████████████|  [ Elapsed: 00:00, Remaining: 00:00 ] (0.106MB)
[1/1] Processing:  100%|████████████████████████████████████████████████████████|  [ Elapsed: 00:02, Remaining: 00:00 ]
      Downloading: 100%|██████████████████████████████████████████████|  [ Elapsed: 00:00, Remaining: 00:00 ] (0.106MB)
<ipython-input-12-8facc2e9008d>:85: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/in

Process for: 2014-10-24 :


[1/1] Processing:  100%|████████████████████████████████████████████████████████|  [ Elapsed: 00:02, Remaining: 00:00 ]
      Downloading: 100%|██████████████████████████████████████████████|  [ Elapsed: 00:00, Remaining: 00:00 ] (0.106MB)
[1/1] Processing:  100%|████████████████████████████████████████████████████████|  [ Elapsed: 00:02, Remaining: 00:00 ]
      Downloading: 100%|██████████████████████████████████████████████|  [ Elapsed: 00:00, Remaining: 00:00 ] (0.106MB)
[1/1] Processing:  100%|████████████████████████████████████████████████████████|  [ Elapsed: 00:02, Remaining: 00:00 ]
      Downloading: 100%|██████████████████████████████████████████████|  [ Elapsed: 00:00, Remaining: 00:00 ] (0.106MB)


Wall time: 8min 38s


In [18]:
TotalSwarmRes_A = pd.concat(listdfa, join='outer', axis=0)
TotalSwarmRes_B = pd.concat(listdfb, join='outer', axis=0)
TotalSwarmRes_C = pd.concat(listdfc, join='outer', axis=0)
TotalSwarmRes_A

,Radius,F_res_CHAOS_MCO_MLI_MMA,B_NEC_res_CHAOS_MCO_MLI_MMA,Latitude,Longitude,Spacecraft,N_res,E_res,C_res,timestamp
epoch,,,,,,,,,,
1408320000,6832723.53,6.514502,"[-2.8183253888937543, 9.260075627217248, 6.498...",66.843847,-134.672531,A,-2.818325,9.260076,6.498977,2014-08-18 00:00:00
1408320060,6832377.42,2.395450,"[-29.73261938928863, 34.44850645428414, 4.3191...",70.654918,-133.562740,A,-29.732619,34.448506,4.319175,2014-08-18 00:01:00
1408320120,6832096.43,-6.841385,"[-72.88157339266945, 62.65663008061824, -3.129...",74.451293,-131.817227,A,-72.881573,62.656630,-3.129330,2014-08-18 00:02:00
1408320180,6831883.56,-13.353861,"[-149.11723897890533, 86.43665049543199, -8.00...",78.218581,-128.828006,A,-149.117239,86.436650,-8.007169,2014-08-18 00:03:00
1408320240,6831741.03,-41.131992,"[4.893829769317335, 2.8739988412246475, -41.35...",81.915833,-122.893911,A,4.893830,2.873999,-41.353059,2014-08-18 00:04:00
...,...,...,...,...,...,...,...,...,...,...
1414194900,6847673.69,-0.987636,"[-2.6561161702029494, 11.664364604459138, 0.25...",-41.301711,126.631396,A,-2.656116,11.664365,0.255692,2014-10-24 23:55:00
1414194960,6847383.70,-1.134736,"[-7.871250423335368, 8.892720838018022, -1.507...",-37.480910,126.676727,A,-7.871250,8.892721,-1.507490,2014-10-24 23:56:00
1414195020,6847058.98,1.910127,"[-10.306672975977563, 8.025024193009472, -6.33...",-33.658749,126.693239,A,-10.306673,8.025024,-6.330879,2014-10-24 23:57:00


## ST-IDW Process for residuals

In [22]:
# 2. Filter Space and time ST-IDW based on GPS points. ST_IDW_Process
# Interpolation of the Swarm Residuals., NEC interpolated residuals for each GPS Point.

# Input:  GPS Track columns, SwarmDataDF
# Output: GPSTrack+ResidualsInterpolated

def ST_IDW_Process (GPSLat,GPSLong,GPSDateTime,GPSTime, DT):
    
    # 1. Runnig the DfTime_func function to filter by the defined Delta Time.
    time_kernel_A = DfTime_func(TotalSwarmRes_A,GPSTime,DT)
    time_kernel_B = DfTime_func(TotalSwarmRes_A,GPSTime,DT)
    time_kernel_C = DfTime_func(TotalSwarmRes_A,GPSTime,DT)
    #2. Computing the dt as the difference between the datetime and the datetime from swarm point. At this point
    #   we have filtered the swarm point by time.
    time_kernel_A['dT'] = (GPSTime - (time_kernel_A.index))
    time_kernel_B['dT'] = (GPSTime - (time_kernel_B.index))
    time_kernel_C['dT'] = (GPSTime - (time_kernel_C.index))
    
    #3.Computing the ds
    # 3.1 Parsing the requieres parameters for distance_to_GPS function
    s_lat = GPSLat; e_lat = time_kernel_A['Latitude']; s_lng = GPSLong; e_lng = time_kernel_A['Longitude']
    
    ## Running the function, based on the previous parameters, for SatA
    time_kernel_A['distance']= distance_to_GPS(s_lat, s_lng, e_lat, e_lng) 
    
    s_lat = GPSLat; e_lat = time_kernel_B['Latitude']; s_lng = GPSLong; e_lng = time_kernel_B['Longitude']  
    # Running the function, based on the previous parameters, for SatB
    time_kernel_B['distance']= distance_to_GPS(s_lat, s_lng, e_lat, e_lng)
    
    s_lat = GPSLat; e_lat = time_kernel_C['Latitude']; s_lng = GPSLong; e_lng = time_kernel_C['Longitude']  
    # Running the function, based on the previous parameters, for SatC
    time_kernel_C['distance']= distance_to_GPS(s_lat, s_lng, e_lat, e_lng) 
    
    #3.2 Computing the R distance.
    time_kernel_A['r']= Kradius(GPSLat)
    time_kernel_B['r']= Kradius(GPSLat)
    time_kernel_C['r']= Kradius(GPSLat)
    
    #3.3 Filtering row that only fall into the computed R value.
    space_time_kA=time_kernel_A[time_kernel_A['distance']<=time_kernel_A['r']]
    space_time_kB=time_kernel_B[time_kernel_B['distance']<=time_kernel_B['r']]
    space_time_kC=time_kernel_C[time_kernel_C['distance']<=time_kernel_C['r']]
    
    #4. Calculating the number of points per satellite that have passed the Space and Time Windows.
    NumPointsA = len(space_time_kA.index)
    NumPointsB = len(space_time_kB.index)
    NumPointsC = len(space_time_kC.index)
    TolSatPts = (NumPointsA+NumPointsB+NumPointsC)
    
    #7. Combining the three satellited messures into a bigger dataframe that store all the Swarm points that were filtered. 
    frames = [space_time_kA, space_time_kB, space_time_kC] #List to index the specific SatId to the new full DF.
    SwarmResiduals_ST_filtered = pd.concat(frames, keys=['A', 'B', 'C'], sort=False)
    
    #8. Computing the minimum distance and the average distance.
    MinDistance = SwarmResiduals_ST_filtered['distance'].min()
    AvDistance = SwarmResiduals_ST_filtered['distance'].mean()
    
    #9. Computing the d (hypotenuse compused from the edges ds, dt values
    ds = SwarmResiduals_ST_filtered['distance']
    r = SwarmResiduals_ST_filtered['r']
    dt = SwarmResiduals_ST_filtered['dT']
    SwarmResiduals_ST_filtered['Dj']= DistJ(ds, r, dt, DT)
   
    #10 Calculating the weigth values based on the previuos parameters.
    SwarmResiduals_ST_filtered['W']= 1/((SwarmResiduals_ST_filtered['Dj'])**2) #We need to make more clear this part.
    
    #11. Computing the Sum of weigths
    SumW = SwarmResiduals_ST_filtered['W'].sum()
    #12. Distribution of weigths
    SwarmResiduals_ST_filtered['Wj'] = SwarmResiduals_ST_filtered['W']/SumW 
    #Could be  some bias. So we need to find a way to apply this weigth maybe base on distance and time using DT (4hrs)
    
    #13. Computing the Magnetic componente based on the weigths prevoius weigths. 
    N_res_int = (SwarmResiduals_ST_filtered['Wj']*SwarmResiduals_ST_filtered['N_res']).sum()
    E_res_int = (SwarmResiduals_ST_filtered['Wj']*SwarmResiduals_ST_filtered['E_res']).sum()
    C_res_int = (SwarmResiduals_ST_filtered['Wj']*SwarmResiduals_ST_filtered['C_res']).sum()

    #15. Write the results into an array that will be a dictionay for the final dataframe.
    resultrowGPS = {'Latitude': GPSLat, 'Longitude': GPSLong, 'DateTime': GPSDateTime, 'N_res': N_res_int, 'E_res': E_res_int, 'C_res':C_res_int, 'TotalPoints':TolSatPts, 'Minimum_Distance':MinDistance, 'Average_Distance':AvDistance}  
    return resultrowGPS

In [21]:
def distance_to_GPS(s_lat, s_lng, e_lat, e_lng): 
    # approximate radius of earth in km
    R = 6373.0
    s_lat = s_lat*(np.pi)/180.0                     
    s_lng = np.deg2rad(s_lng)     
    e_lat = np.deg2rad(e_lat)                       
    e_lng = np.deg2rad(e_lng)
    d = np.sin((e_lat - s_lat)/2)**2 + np.cos(s_lat)*np.cos(e_lat) * np.sin((e_lng - s_lng)/2)**2
    return 2 * R * np.arcsin(np.sqrt(d))

def Kradius (lat):
    if 0 <= lat < 90 :
        #for Northern Latitudes 
        nlat = (-10 * lat) + 1800
        #print("The R on the North")
    if -90 < lat < 0:
        #for Southern Latitudes
        nlat = (10 * lat) + 1800
        #print ("The R on the South")
    return nlat

def DistJ(ds, r, dt, DT):
    eDist = np.sqrt(((ds/r)**2 + (dt/DT)**2)/2)
    return eDist

def DfTime_func (SwarmData, GPSTime, DT):
#     DT= 14400 #deltaT of 14400 secs, 4 hours, if the Period is 1s. 1440 for 10s of period. 240 for data every 60s.
    DataFrame_Per_Time = []
    for index in SwarmData.index:
        if index == GPSTime:
            DataFrame_Per_Time = pd.DataFrame(SwarmData.loc[index-DT:index+DT])
    return DataFrame_Per_Time

In [23]:
%%time
#Sequential mode, applying a traditional loop using iterrows.
DT= 14400
if __name__ == '__main__':
    dn = [] ## List used to add all the GPS points with the annotated MAG Data. See the last bullet point of this process        
    for index, row in GPSData.iterrows():
        GPSLat = row['gpsLat']
        GPSLong = row['gpsLong']
        GPSDateTime = row['gpsDateTime']
        GPSTime = row['epoch']
        print("Process for:", index,"Epoch:",GPSDateTime)
        result=ST_IDW_Process(GPSLat,GPSLong,GPSDateTime,GPSTime, DT)
        dn.append(result)

Process for: 0 Epoch: 2014-09-08 05:54:00
Process for: 1 Epoch: 2014-09-08 06:10:00
Process for: 2 Epoch: 2014-09-08 06:26:00
Process for: 3 Epoch: 2014-09-08 06:42:00
Process for: 4 Epoch: 2014-09-08 07:14:00
Process for: 5 Epoch: 2014-09-08 07:30:00
Process for: 6 Epoch: 2014-09-08 07:46:00
Process for: 7 Epoch: 2014-09-08 08:03:00
Process for: 8 Epoch: 2014-09-08 08:34:00
Process for: 9 Epoch: 2014-09-08 08:50:00
Process for: 10 Epoch: 2014-09-08 09:37:00
Process for: 11 Epoch: 2014-09-08 09:54:00
Process for: 12 Epoch: 2014-09-08 13:53:00
Process for: 13 Epoch: 2014-09-08 14:09:00
Process for: 14 Epoch: 2014-09-08 14:25:00
Process for: 15 Epoch: 2014-09-08 14:41:00
Process for: 16 Epoch: 2014-09-08 15:13:00
Process for: 17 Epoch: 2014-09-08 15:29:00
Process for: 18 Epoch: 2014-09-08 15:45:00
Process for: 19 Epoch: 2014-09-08 16:01:00
Process for: 20 Epoch: 2014-09-08 16:33:00
Process for: 21 Epoch: 2014-09-08 16:49:00
Process for: 22 Epoch: 2014-09-08 17:05:00
Process for: 23 Epoch

Process for: 189 Epoch: 2014-10-04 07:53:00
Process for: 190 Epoch: 2014-08-30 12:39:00
Process for: 191 Epoch: 2014-08-30 14:42:00
Process for: 192 Epoch: 2014-08-30 18:49:00
Process for: 193 Epoch: 2014-09-27 10:54:00
Process for: 194 Epoch: 2014-09-27 11:09:00
Process for: 195 Epoch: 2014-09-27 11:25:00
Process for: 196 Epoch: 2014-09-27 11:40:00
Process for: 197 Epoch: 2014-09-27 11:56:00
Process for: 198 Epoch: 2014-09-27 12:11:00
Process for: 199 Epoch: 2014-09-27 12:27:00
Process for: 200 Epoch: 2014-09-29 22:56:00
Process for: 201 Epoch: 2014-09-30 00:59:00
Process for: 202 Epoch: 2014-09-30 10:44:00
Process for: 203 Epoch: 2014-09-30 11:00:00
Process for: 204 Epoch: 2014-09-30 11:15:00
Process for: 205 Epoch: 2014-09-30 11:56:00
Process for: 206 Epoch: 2014-09-30 12:11:00
Process for: 207 Epoch: 2014-10-01 08:10:00
Process for: 208 Epoch: 2014-10-01 08:25:00
Process for: 209 Epoch: 2014-10-01 08:41:00
Process for: 210 Epoch: 2014-10-01 08:56:00
Process for: 211 Epoch: 2014-10-

Process for: 378 Epoch: 2014-10-01 11:54:00
Process for: 379 Epoch: 2014-10-01 12:10:00
Process for: 380 Epoch: 2014-08-18 06:00:00
Process for: 381 Epoch: 2014-08-18 06:30:00
Process for: 382 Epoch: 2014-08-18 07:00:00
Process for: 383 Epoch: 2014-08-18 07:30:00
Process for: 384 Epoch: 2014-08-18 08:00:00
Process for: 385 Epoch: 2014-08-18 08:30:00
Process for: 386 Epoch: 2014-08-18 09:00:00
Process for: 387 Epoch: 2014-08-18 09:30:00
Process for: 388 Epoch: 2014-08-18 10:00:00
Process for: 389 Epoch: 2014-09-13 10:30:00
Process for: 390 Epoch: 2014-09-13 11:00:00
Process for: 391 Epoch: 2014-09-13 11:30:00
Process for: 392 Epoch: 2014-09-13 12:00:00
Process for: 393 Epoch: 2014-09-13 12:30:00
Process for: 394 Epoch: 2014-09-13 13:00:00
Process for: 395 Epoch: 2014-09-13 13:30:00
Process for: 396 Epoch: 2014-09-13 14:00:00
Process for: 397 Epoch: 2014-09-13 14:30:00
Process for: 398 Epoch: 2014-09-13 15:00:00
Process for: 399 Epoch: 2014-09-13 15:30:00
Process for: 400 Epoch: 2014-09-

Process for: 564 Epoch: 2014-10-01 13:30:00
Process for: 565 Epoch: 2014-10-01 14:00:00
Process for: 566 Epoch: 2014-10-01 14:30:00
Process for: 567 Epoch: 2014-10-01 15:00:00
Process for: 568 Epoch: 2014-10-01 15:30:00
Process for: 569 Epoch: 2014-10-01 16:00:00
Process for: 570 Epoch: 2014-10-01 16:30:00
Process for: 571 Epoch: 2014-10-01 17:00:00
Process for: 572 Epoch: 2014-08-27 21:30:00
Process for: 573 Epoch: 2014-08-27 22:00:00
Process for: 574 Epoch: 2014-08-27 22:30:00
Process for: 575 Epoch: 2014-08-27 23:00:00
Process for: 576 Epoch: 2014-08-27 23:30:00
Process for: 577 Epoch: 2014-08-28 00:00:00
Process for: 578 Epoch: 2014-08-28 00:30:00
Process for: 579 Epoch: 2014-08-28 01:00:00
Process for: 580 Epoch: 2014-08-28 01:30:00
Process for: 581 Epoch: 2014-08-28 02:00:00
Process for: 582 Epoch: 2014-08-28 02:30:00
Process for: 583 Epoch: 2014-08-28 03:00:00
Process for: 584 Epoch: 2014-08-28 03:30:00
Process for: 585 Epoch: 2014-08-28 04:00:00
Process for: 586 Epoch: 2014-08-

Process for: 753 Epoch: 2014-09-05 14:30:00
Process for: 754 Epoch: 2014-09-05 15:00:00
Process for: 755 Epoch: 2014-09-05 15:30:00
Process for: 756 Epoch: 2014-09-05 16:00:00
Process for: 757 Epoch: 2014-09-05 16:30:00
Process for: 758 Epoch: 2014-09-05 17:00:00
Process for: 759 Epoch: 2014-09-05 17:30:00
Process for: 760 Epoch: 2014-09-05 18:00:00
Process for: 761 Epoch: 2014-09-05 18:30:00
Process for: 762 Epoch: 2014-09-05 19:00:00
Process for: 763 Epoch: 2014-09-05 19:30:00
Process for: 764 Epoch: 2014-09-05 20:00:00
Process for: 765 Epoch: 2014-09-05 20:30:00
Process for: 766 Epoch: 2014-09-05 21:00:00
Process for: 767 Epoch: 2014-09-05 21:30:00
Process for: 768 Epoch: 2014-09-05 22:00:00
Process for: 769 Epoch: 2014-09-05 22:30:00
Process for: 770 Epoch: 2014-09-05 23:00:00
Process for: 771 Epoch: 2014-09-05 23:30:00
Process for: 772 Epoch: 2014-09-06 00:00:00
Process for: 773 Epoch: 2014-09-06 00:30:00
Process for: 774 Epoch: 2014-09-06 01:00:00
Process for: 775 Epoch: 2014-09-

Process for: 942 Epoch: 2014-10-24 08:00:00
Process for: 943 Epoch: 2014-10-24 08:30:00
Process for: 944 Epoch: 2014-10-24 09:00:00
Process for: 945 Epoch: 2014-10-02 13:11:00
Process for: 946 Epoch: 2014-10-02 13:45:00
Process for: 947 Epoch: 2014-10-02 18:47:00
Process for: 948 Epoch: 2014-10-02 19:19:00
Process for: 949 Epoch: 2014-10-03 00:19:00
Process for: 950 Epoch: 2014-10-03 03:00:00
Process for: 951 Epoch: 2014-09-23 18:08:00
Process for: 952 Epoch: 2014-09-30 07:36:00
Process for: 953 Epoch: 2014-09-30 18:35:00
Process for: 954 Epoch: 2014-10-01 05:19:00
Process for: 955 Epoch: 2014-10-01 16:01:00
Process for: 956 Epoch: 2014-10-02 02:45:00
Process for: 957 Epoch: 2014-10-10 08:30:00
Process for: 958 Epoch: 2014-10-10 13:30:00
Process for: 959 Epoch: 2014-10-15 08:30:00
Process for: 960 Epoch: 2014-10-15 13:30:00
Process for: 961 Epoch: 2014-10-22 13:31:00
Process for: 962 Epoch: 2014-10-23 08:30:00
Process for: 963 Epoch: 2014-10-23 13:30:00
Process for: 964 Epoch: 2014-10-

In [24]:
GPS_ResInt = pd.DataFrame(dn)
# GPS_ResInt.DateTime[3].year
GPS_ResInt.head()

,Latitude,Longitude,DateTime,N_res,E_res,C_res,TotalPoints,Minimum_Distance,Average_Distance
0,70.854717,68.307333,2014-09-08 05:54:00,-14.993330,7.097887,2.737431,24,459.037634,621.317068
1,70.830300,67.975050,2014-09-08 06:10:00,-14.641426,6.633903,2.777449,24,451.300487,621.749880
2,70.761717,67.752417,2014-09-08 06:26:00,-14.171211,6.038794,2.822754,24,449.951494,622.840600
3,70.686517,67.561983,2014-09-08 06:42:00,-13.613031,5.453029,2.846921,24,450.187612,624.112300
4,70.685450,67.548317,2014-09-08 07:14:00,-12.669296,4.684420,2.840456,24,449.891623,624.140092


## CHAOS Ground Values

In [ ]:
# 3. Compute the ground CHAOS Values for the previous GPS Track. Ionosphere_correction_results
#    Add the residuals to the computed CHAOS Values.
# Input:  GPSTrack+ResidualsInterpolated
# Output: AnnotatedGPSTrack

In [27]:
def CHAOS_ground_values(GPS_ResInt):
    
    model = load_CHAOS_matfile('C:\\foss4guk19-jupyter-master\\Project_StAndrews\\Parallel\\CHAOS-7.mat')
    theta = 90-GPS_ResInt['Latitude'].values
    phi = GPS_ResInt['Longitude'].values
    rad_geoc_ground, theta_geoc_ground, sd_ground, cd_ground = gg_to_geo(0, theta)
    time= mjd2000(pd.DatetimeIndex(GPS_ResInt['DateTime']).year, pd.DatetimeIndex(GPS_ResInt['DateTime']).month, pd.DatetimeIndex(GPS_ResInt['DateTime']).day)

    B_r_core, B_t_core, B_phi_core = model.synth_values_tdep(time, rad_geoc_ground, theta_geoc_ground, phi) #Core Contribution
    B_r_crust, B_t_crust, B_phi_crust = model.synth_values_static(rad_geoc_ground, theta_geoc_ground, phi) #Crust Contribution
    B_r_magneto, B_t_magneto, B_phi_magneto = model.synth_values_gsm(time, rad_geoc_ground, theta_geoc_ground, phi) #Magnetosphere contribution.

    #Change the direcction of the axis from XYZ to r,theta and phi.
    B_r_swarm, B_t_swarm, B_phi_swarm = -GPS_ResInt['C_res'], -GPS_ResInt['N_res'], GPS_ResInt['E_res']


    #Compute the magnetic component (r,theta,phi) at ground level.
    B_r_ground = B_r_core + B_r_crust + B_r_magneto + B_r_swarm #(-Z)
    B_t_ground = B_t_core + B_t_crust + B_t_magneto + B_t_swarm #(-X)
    B_phi_ground = B_phi_core + B_phi_crust +B_phi_magneto + B_phi_swarm #(Y)

    #Convert B_r_, B_t_, and B_phi to XYZ (NEC)
    Z_chaos = -B_r_ground   #Z
    X_chaos = -B_t_ground   #X
    Y_chaos = B_phi_ground  #Y

    #Rotate the X(N) and Z(C) magnetic field values of the chaos models into the geodectic frame using the sd and cd (sine and cosine d from gg_to_geo) 
    X_obs = X_chaos*cd_ground + Z_chaos*sd_ground #New N
    Z_obs = Z_chaos*cd_ground - X_chaos*sd_ground #New C
    Y_obs = Y_chaos # New E
    return X_obs, Y_obs, Z_obs

In [28]:
X_obs, Y_obs, Z_obs =CHAOS_ground_values(GPS_ResInt)
GPS_ResInt['N'] =pd.Series(X_obs)
GPS_ResInt['E'] =pd.Series(Y_obs)
GPS_ResInt['C'] =pd.Series(Z_obs)
GPS_ResInt.head()

,Latitude,Longitude,DateTime,N_res,E_res,C_res,TotalPoints,Minimum_Distance,Average_Distance,N,E,C
0,70.854717,68.307333,2014-09-08 05:54:00,-14.993330,7.097887,2.737431,24,459.037634,621.317068,6948.878113,3848.316449,57704.851845
1,70.830300,67.975050,2014-09-08 06:10:00,-14.641426,6.633903,2.777449,24,451.300487,621.749880,6985.748607,3865.543602,57657.233192
2,70.761717,67.752417,2014-09-08 06:26:00,-14.171211,6.038794,2.822754,24,449.951494,622.840600,7033.205734,3878.348668,57624.777967
3,70.686517,67.561983,2014-09-08 06:42:00,-13.613031,5.453029,2.846921,24,450.187612,624.112300,7080.834506,3889.295760,57597.392276
4,70.685450,67.548317,2014-09-08 07:14:00,-12.669296,4.684420,2.840456,24,449.891623,624.140092,7083.276976,3889.178351,57595.394154


## Compute Magnetic Values based on new NEC Ground Values

In [31]:
#14. Having Intepolated and weigth magnetic values, we can compute the other magnectic components. 
GPS_ResInt['H'] = np.sqrt((GPS_ResInt['N']**2)+(GPS_ResInt['E']**2))
#check the arcgtan in python., From arctan2 is saver.
DgpsRad = np.arctan2(GPS_ResInt['E'],GPS_ResInt['N'])
GPS_ResInt['D'] = np.degrees(DgpsRad)
GPS_ResInt['F'] = np.sqrt((GPS_ResInt['N']**2)+(GPS_ResInt['E']**2)+(GPS_ResInt['C']**2))
IgpsRad = np.arctan2(GPS_ResInt['C'],GPS_ResInt['H'])
GPS_ResInt['I'] = np.degrees(IgpsRad)
GPS_ResInt

,Latitude,Longitude,DateTime,N_res,E_res,C_res,TotalPoints,Minimum_Distance,Average_Distance,N,E,C,H,D,F,I
0,70.854717,68.307333,2014-09-08 05:54:00,-14.993330,7.097887,2.737431,24,459.037634,621.317068,6948.878113,3848.316449,57704.851845,7943.327169,28.977859,58249.003193,82.162241
1,70.830300,67.975050,2014-09-08 06:10:00,-14.641426,6.633903,2.777449,24,451.300487,621.749880,6985.748607,3865.543602,57657.233192,7983.928290,28.957821,58207.384843,82.116257
2,70.761717,67.752417,2014-09-08 06:26:00,-14.171211,6.038794,2.822754,24,449.951494,622.840600,7033.205734,3878.348668,57624.777967,8031.660556,28.873835,58181.806495,82.065312
3,70.686517,67.561983,2014-09-08 06:42:00,-13.613031,5.453029,2.846921,24,450.187612,624.112300,7080.834506,3889.295760,57597.392276,8078.665658,28.778708,58161.193556,82.015726
4,70.685450,67.548317,2014-09-08 07:14:00,-12.669296,4.684420,2.840456,24,449.891623,624.140092,7083.276976,3889.178351,57595.394154,8080.750025,28.769641,58159.504372,82.013419
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
968,55.214750,22.509733,2014-10-01 10:37:00,10.807193,-5.016359,-5.022437,24,483.161878,866.742201,16979.935221,1824.188049,47880.418942,17077.642172,6.131881,50834.834318,70.370066
969,53.392167,7.292517,2014-10-03 11:28:00,2.772405,-2.665188,-5.837464,15,183.224731,570.361318,18344.700978,501.757660,45910.231691,18351.561642,1.566743,49442.180258,68.212073
970,53.107067,5.922067,2014-10-04 07:11:00,-2.558009,-6.859043,-6.696689,18,371.831436,753.975771,18527.386454,336.804203,45665.764201,18530.447536,1.041449,49282.243313,67.913461
971,53.014967,5.811383,2014-10-04 14:12:00,3.057266,-11.001720,-8.466342,6,1210.876757,1224.256745,18583.170400,321.496760,45609.827558,18585.951207,0.991143,49251.334521,67.829162


In [32]:
#Exporting the CSV file
os.chdir(r"../results")
outputfile ="GeoMagV2_"+gpsfilename
export_csv = GPS_ResInt.to_csv (outputfile, index = None, header=True)